In [3]:
import numpy as np
import torch
import torch.optim as optim
torch.set_printoptions(edgeitems=2, linewidth=75)


In [4]:
t_c = torch.tensor([0.5, 14.0, 15.0, 28.0, 11.0,
                    8.0, 3.0, -4.0, 6.0, 13.0, 21.0])
t_u = torch.tensor([35.7, 55.9, 58.2, 81.9, 56.3, 48.9,
                    33.9, 21.8, 48.4, 60.4, 68.4])
#Normalizing (value between -1 - 1)
t_un = 0.01 * t_u

In [5]:
def model(t_u, w1, w2, b):
    return w2 * t_u ** 2 + w1 * t_u + b

In [6]:
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()


In [14]:

n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

train_indices, val_indices  # <1>

train_t_u = t_u[train_indices]
train_t_c = t_c[train_indices]

val_t_u = t_u[val_indices]
val_t_c = t_c[val_indices]

train_t_un = 0.01 * train_t_u
val_t_un = 0.01 * val_t_u

In [15]:
def training_loop(n_epochs, optimizer, params, train_t_u, val_t_u,
                  train_t_c, val_t_c):
    for epoch in range(1, n_epochs + 1):
        train_t_p = model(train_t_u, *params) # <1>
        train_loss = loss_fn(train_t_p, train_t_c)
                             
        val_t_p = model(val_t_u, *params) # <1>
        val_loss = loss_fn(val_t_p, val_t_c)
        
        optimizer.zero_grad()
        train_loss.backward() # <2>
        optimizer.step()

        if epoch <= 3 or epoch % 500 == 0:
            print(f"Epoch {epoch}, Training loss {train_loss.item():.4f},"
                  f" Validation loss {val_loss.item():.4f}")
            
    return params

In [16]:
#Non-Linear Model with SGD reserving 20% of the data for validation
params = torch.tensor([1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-2
optimizer = optim.SGD([params], lr=learning_rate)

training_loop(
    n_epochs = 5000, 
    optimizer = optimizer,
    params = params,
    train_t_u = train_t_un, # <1> 
    val_t_u = val_t_un, # <1> 
    train_t_c = train_t_c,
    val_t_c = val_t_c)

Epoch 1, Training loss 124.1280, Validation loss 351.3965
Epoch 2, Training loss 119.4730, Validation loss 343.2898
Epoch 3, Training loss 115.0613, Validation loss 335.5335
Epoch 500, Training loss 22.9361, Validation loss 93.0040
Epoch 1000, Training loss 14.8027, Validation loss 56.8464
Epoch 1500, Training loss 9.8759, Validation loss 34.9428
Epoch 2000, Training loss 6.8915, Validation loss 21.6756
Epoch 2500, Training loss 5.0835, Validation loss 13.6412
Epoch 3000, Training loss 3.9882, Validation loss 8.7769
Epoch 3500, Training loss 3.3246, Validation loss 5.8335
Epoch 4000, Training loss 2.9224, Validation loss 4.0537
Epoch 4500, Training loss 2.6786, Validation loss 2.9787
Epoch 5000, Training loss 2.5307, Validation loss 2.3305


tensor([24.7269, 24.6632, -9.3986], requires_grad=True)

In [21]:
#Non-Linear Model with Adam reserving 20% of the data for validation
params = torch.tensor([1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-2
optimizer = optim.Adam([params], lr=learning_rate)

training_loop(
    n_epochs = 5000, 
    optimizer = optimizer,
    params = params,
    train_t_u = train_t_un, # <1> 
    val_t_u = val_t_un, # <1> 
    train_t_c = train_t_c,
    val_t_c = val_t_c)

Epoch 1, Training loss 124.1280, Validation loss 351.3965
Epoch 2, Training loss 123.7794, Validation loss 350.7367
Epoch 3, Training loss 123.4316, Validation loss 350.0777
Epoch 500, Training loss 37.4905, Validation loss 158.8574
Epoch 1000, Training loss 27.1680, Validation loss 111.6235
Epoch 1500, Training loss 23.7155, Validation loss 93.1775
Epoch 2000, Training loss 19.9536, Validation loss 76.3038
Epoch 2500, Training loss 16.0662, Validation loss 59.1560
Epoch 3000, Training loss 12.3658, Validation loss 42.9948
Epoch 3500, Training loss 9.1191, Validation loss 28.9970
Epoch 4000, Training loss 6.5085, Validation loss 17.9389
Epoch 4500, Training loss 4.6080, Validation loss 10.0965
Epoch 5000, Training loss 3.3809, Validation loss 5.2456


tensor([19.0104, 26.2467, -6.8815], requires_grad=True)